In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Dropout,Dense,Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
import numpy as np
import os
from sklearn.metrics import accuracy_score

In [11]:
batch = 32

train_imgdatagen = ImageDataGenerator( rescale=1./255,
                                      rotation_range = 10,
                                      width_shift_range = 0.1,
                                      height_shift_range = 0.1,
                                      zoom_range = 0.2,
                                      horizontal_flip = True
                                    )
test_imagedatagen = ImageDataGenerator(rescale=1./255)

train_data = train_imgdatagen.flow_from_directory(
    'images/train/',
    target_size = (300,300),
    color_mode='grayscale',
    batch_size = batch,
    classes = ['NONE','ONE','TWO','THREE','FOUR','FIVE'],
    class_mode = 'categorical'
    )
test_data = test_imagedatagen.flow_from_directory(
    'images/test/',
    target_size = (300,300),
    color_mode='grayscale',
    batch_size = 3600,
    classes = ['NONE','ONE','TWO','THREE','FOUR','FIVE'],
    class_mode = 'categorical'
    )

Found 9081 images belonging to 6 classes.
Found 3632 images belonging to 6 classes.


In [3]:
train_data.class_indices == test_data.class_indices

True

In [7]:
for X,y in train_data:
    print(X.shape, y.shape)
    
    break
    
print(len(train_data))

(32, 300, 300, 1) (32, 6)
284


In [8]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation='relu', input_shape=(300,300,1)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(6,activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 298, 298, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 147, 147, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 71, 71, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 33, 33, 128)       147584    
__________

In [12]:
model.compile(optimizer = 'adadelta',loss = 'categorical_crossentropy',metrics= ['acc'])

In [9]:
callbacks_list = [
    EarlyStopping(monitor='val_loss',patience=10),
    ModelCheckpoint(filepath = 'model_finger_count3.h5',monitor='val_loss',save_best_only=True)
]

In [13]:
model.load_weights('model_finger_count1.h5')

In [14]:
history = model.fit_generator(
    train_data,
    steps_per_epoch = 64,
    epochs = 10,
    validation_data = test_data,
    validation_steps = 28,
    callbacks=callbacks_list
)

Epoch 1/10
64/64 [==============================] - 184s 3s/step - loss: 0.0825 - acc: 0.9736 - val_loss: 0.0064 - val_acc: 0.9972
Epoch 2/10
64/64 [==============================] - 166s 3s/step - loss: 0.0632 - acc: 0.9800 - val_loss: 0.0058 - val_acc: 0.9983
Epoch 3/10
64/64 [==============================] - 164s 3s/step - loss: 0.0556 - acc: 0.9839 - val_loss: 0.0037 - val_acc: 0.9994
Epoch 4/10
64/64 [==============================] - 162s 3s/step - loss: 0.0347 - acc: 0.9907 - val_loss: 8.8552e-04 - val_acc: 1.0000
Epoch 5/10
64/64 [==============================] - 163s 3s/step - loss: 0.0428 - acc: 0.9873 - val_loss: 0.0090 - val_acc: 0.9970
Epoch 6/10
64/64 [==============================] - 155s 2s/step - loss: 0.0517 - acc: 0.9844 - val_loss: 0.0058 - val_acc: 0.9981
Epoch 7/10
64/64 [==============================] - 159s 2s/step - loss: 0.0469 - acc: 0.9858 - val_loss: 0.0073 - val_acc: 0.9978
Epoch 8/10
64/64 [==============================] - 157s 2s/step - loss: 0.0321

In [19]:
model = load_model('model_finger_count3.h5')

In [43]:
X_test = test_data[0][0]
y_test = test_data[0][1]
pred = model.predict(X_test)


In [44]:
y_true = np.argmax(y_test,axis=1)
y_pred = np.argmax(pred,axis=1)
accuracy_score(y_true,y_pred)


1.0

In [45]:
true_match = (y_true==y_pred)
print('Accuracy : %f' % (np.sum(true_match)/true_match.shape[0]*100))

Accuracy : 100.000000
